In [ ]:
Cell 1 – Setup & Imports
!pip install torch torchvision matplotlib
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os


In [ ]:
Cell 2 – Data Loading
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToTensor()
])

train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transforms)
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

print("Training data shape:", train_data.data.shape)
print("Test data shape:", test_data.data.shape)


In [ ]:
Cell 3 – Display a Sample
image, label = test_data[0]
plt.imshow(image.permute(1, 2, 0))
plt.title(f"Label: {label}")
plt.show()


In [ ]:
Cell 4 – Model Definition
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        return self.network(x)


In [ ]:
Cell 5 – Training Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ImageClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
train_losses = []


In [ ]:
Cell 6 – Training Loop
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(trainloader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


In [ ]:
Cell 7 – Plot Loss
plt.plot(range(epochs), train_losses)
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.title('Training Loss per Epoch')
plt.show()


In [ ]:
Cell 8 – Evaluate Accuracy
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

if accuracy >= 45:
    print("✅ Model meets the performance target. Recommendation: Build in-house.")
else:
    print("⚠️ Model below target. Consider using a pre-trained or commercial solution.")


In [ ]:
Cell 9 – Save Model
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/model.pth")
print("Model saved successfully.")
